In [9]:
# These numbers come from the D1_merged_all_cracks_6micron_griddata.vtk file in /data. They are 1 less than the 
# DIMENSIONS specified because we are using the center of each voxel. 
import numpy as np

def get_voxel_centers_as_array(num_x_pts, num_y_pts, num_z_pts, increment, file):
    total_voxels = num_x_pts * num_y_pts * num_z_pts
    natee = 0
    next_print = total_voxels / 10
    
    voxel_centers = np.empty((num_x_pts, num_y_pts, num_z_pts), dtype=int)
    with open(file, 'r') as voxel_centers_file:
        voxel_centers_file.readline()
        for line in voxel_centers_file:
            tokens = line.split(',')
            x = int(float(tokens[0]))
            y = int(float(tokens[1]))
            z = int(float(tokens[2]))
            grain_id = int(tokens[3])
            # Divide x,y,z because we want to fill every bucket of the array
            voxel_centers[x // increment][y // increment][z // increment] = grain_id
            
            # Print progress
            natee += 1
            if natee >= next_print:
                print('Progress: ', natee / total_voxels)
                next_print += total_voxels / 10
        
    return voxel_centers

In [10]:
# This will give the nearest voxel center (x,y,z) based on the voxel size being the length of 1 side of the voxel
def find_nearest_voxel_center(x, y, z, voxel_size):
    voxel_x = int(((x + (voxel_size / 2)) // voxel_size) * voxel_size)
    voxel_y = int(((y + (voxel_size / 2)) // voxel_size) * voxel_size)
    voxel_z = int(((z + (voxel_size / 2)) // voxel_size) * voxel_size)
    return voxel_x, voxel_y, voxel_z

In [11]:
# Reads in the crack_front_points file. dadN3Dline is the output we're concerned with. Gives a list.
def get_crack_front_points(input_file):
    with open(input_file, 'r') as crack_points_file:
        crack_points_file.readline()
        crack_points = []
        for line in crack_points_file:
            tokens = line.split(',')
            crack_id = tokens[0]
            theta = tokens[1]
            x = tokens[5]
            y = tokens[6]
            z = tokens[7].strip()
            dadN3Dline = tokens[3]
            if int(crack_id) > 0:
                crack_points.append((crack_id, theta, x, y, z, dadN3Dline))
    return crack_points

In [117]:
# Given an array of the voxel centers arr[x][y][z] = grain_id at those 'coordinates / voxel_size'. Returns the nearest 
# voxel center that has a different grain_id associated with it. Basically, just does a breadth-first search...
from collections import deque

def is_border_grain(gids, x, y, z, center_gid):
    center_gid = get_actual_grain_id(center_gid)
    if get_actual_grain_id(gids[x][y][z]) == center_gid:
        return False
    if x+1 < len(gids) and get_actual_grain_id(gids[x+1][y][z]) == center_gid:
        return True
    if x > 0 and get_actual_grain_id(gids[x-1][y][z]) == center_gid:
        return True
    if y+1 < len(gids[0]) and get_actual_grain_id(gids[x][y+1][z]) == center_gid:
        return True
    if y > 0 and get_actual_grain_id(gids[x][y-1][z]) == center_gid:
        return True
    if z+1 < len(gids[0][0]) and get_actual_grain_id(gids[x][y][z+1]) == center_gid:
        return True
    if z > 0 and get_actual_grain_id(gids[x][y][z-1]) == center_gid:
        return True
    return False

def find_nearest_grain_boundary(voxel_centers, checked_voxels, x, y, z, grain_id):
    # Shoot out feelers
    x_pos_feeler_x = x
    x_pos_feeler_y = y
    x_pos_feeler_z = z
    
    x_neg_feeler_x = x
    x_neg_feeler_y = y
    x_neg_feeler_z = z

    y_pos_feeler_x = x
    y_pos_feeler_y = y
    y_pos_feeler_z = z
    
    y_neg_feeler_x = x
    y_neg_feeler_y = y
    y_neg_feeler_z = z
    
    z_pos_feeler_x = x
    z_pos_feeler_y = y
    z_pos_feeler_z = z
    
    z_neg_feeler_x = x
    z_neg_feeler_y = y
    z_neg_feeler_z = z
    
    grain_id = get_actual_grain_id(grain_id)

    while x_pos_feeler_x+1 < len(voxel_centers) and get_actual_grain_id(voxel_centers[x_pos_feeler_x][x_pos_feeler_y][x_pos_feeler_z]) == grain_id:
        x_pos_feeler_x += 1

    while x_neg_feeler_x > 0 and get_actual_grain_id(voxel_centers[x_neg_feeler_x][x_neg_feeler_y][x_neg_feeler_z]) == grain_id:
        x_neg_feeler_x -= 1

    while y_pos_feeler_y+1 < len(voxel_centers[0]) and get_actual_grain_id(voxel_centers[y_pos_feeler_x][y_pos_feeler_y][y_pos_feeler_z]) == grain_id:
        y_pos_feeler_y += 1

    while y_neg_feeler_y > 0 and get_actual_grain_id(voxel_centers[y_neg_feeler_x][y_neg_feeler_y][y_neg_feeler_z]) == grain_id:
        y_neg_feeler_y -= 1
        
    while z_pos_feeler_z+1 < len(voxel_centers[0]) and get_actual_grain_id(voxel_centers[z_pos_feeler_x][z_pos_feeler_y][z_pos_feeler_z]) == grain_id:
        z_pos_feeler_z += 1

    while z_neg_feeler_z > 0 and get_actual_grain_id(voxel_centers[z_neg_feeler_x][z_neg_feeler_y][z_neg_feeler_z]) == grain_id:
        z_neg_feeler_z -= 1
    
    queue = deque()
    
    # Test if they found a different grain, add them to queue if so
    used_x_pos = False
    used_x_neg = False
    used_y_pos = False
    used_y_neg = False
    used_z_pos = False
    used_z_neg = False

    if get_actual_grain_id(voxel_centers[x_pos_feeler_x][x_pos_feeler_y][x_pos_feeler_z]) != grain_id:
        queue.append((x_pos_feeler_x, x_pos_feeler_y, x_pos_feeler_z))
        used_x_pos = True
        
    if get_actual_grain_id(voxel_centers[x_neg_feeler_x][x_neg_feeler_y][x_neg_feeler_z]) != grain_id:
        queue.append((x_neg_feeler_x, x_neg_feeler_y, x_neg_feeler_z))
        used_x_neg = True
        
    if get_actual_grain_id(voxel_centers[y_pos_feeler_x][y_pos_feeler_y][y_pos_feeler_z]) != grain_id:
        queue.append((y_pos_feeler_x, y_pos_feeler_y, y_pos_feeler_z))
        used_y_pos = True
    
    if get_actual_grain_id(voxel_centers[y_neg_feeler_x][y_neg_feeler_y][y_neg_feeler_z]) != grain_id:
        queue.append((y_neg_feeler_x, y_neg_feeler_y, y_neg_feeler_z))
        used_y_neg = True

    if get_actual_grain_id(voxel_centers[z_pos_feeler_x][z_pos_feeler_y][z_pos_feeler_z]) != grain_id:
        queue.append((z_pos_feeler_x, z_pos_feeler_y, z_pos_feeler_z))
        used_z_pos = True
    
    if get_actual_grain_id(voxel_centers[z_neg_feeler_x][z_neg_feeler_y][z_neg_feeler_z]) != grain_id:
        queue.append((z_neg_feeler_x, z_neg_feeler_y, z_neg_feeler_z))
        used_z_neg = True
        
    print(queue)
    
    min_distance_so_far = 99999999
    min_x = -999999999
    min_y = -999999999
    min_z = -999999999
    min_grain_id = 999999999
    
    natee = 0
    while len(queue) > 0:
        u = queue.popleft()
        current_x, current_y, current_z = u
        current_distance = ((current_x - x)**2 + (current_y - y)**2 + (current_z - z)**2)**.5
        if current_distance < min_distance_so_far:
            # TODO: If this is in front of the crack front in both the X and Y directions?
            min_distance_so_far = current_distance
            min_x = current_x
            min_y = current_y
            min_z = current_z
            min_grain_id = get_actual_grain_id(voxel_centers[min_x][min_y][min_z])
            
        checked_voxels[current_x][current_y][current_z] = True
    
        # Check diagonals as well as adjacents here
        if current_x+1 < len(voxel_centers) and not checked_voxels[current_x+1][current_y][current_z] and \
                is_border_grain(voxel_centers, current_x+1, current_y, current_z, grain_id):
            queue.append((current_x+1, current_y, current_z))
            checked_voxels[current_x+1][current_y][current_z] = True
            
        if current_x > 0 and not checked_voxels[current_x-1][current_y][current_z] and \
                is_border_grain(voxel_centers, current_x-1, current_y, current_z, grain_id):
            queue.append((current_x-1, current_y, current_z))
            checked_voxels[current_x-1][current_y][current_z] = True
            
        if current_y+1 < len(voxel_centers[0]) and not checked_voxels[current_x][current_y+1][current_z] and \
                is_border_grain(voxel_centers, current_x, current_y+1, current_z, grain_id):
            queue.append((current_x, current_y+1, current_z))
            checked_voxels[current_x][current_y+1][current_z] = True
            
        if current_y > 0 and not checked_voxels[current_x][current_y-1][current_z] and \
                is_border_grain(voxel_centers, current_x, current_y-1, current_z, grain_id):
            queue.append((current_x, current_y-1, current_z))
            checked_voxels[current_x][current_y-1][current_z] = True
            
        if current_z+1 < len(voxel_centers[0][0]) and not checked_voxels[current_x][current_y][current_z+1] and \
                is_border_grain(voxel_centers, current_x, current_y, current_z+1, grain_id):
            queue.append((current_x, current_y, current_z+1))
            checked_voxels[current_x][current_y][current_z+1] = True
            
        if current_z > 0 and not checked_voxels[current_x][current_y][current_z-1] and \
                is_border_grain(voxel_centers, current_x, current_y, current_z-1, grain_id):
            queue.append((current_x, current_y, current_z-1))
            checked_voxels[current_x][current_y][current_z-1] = True
            
        # X and Y
        if current_x+1 < len(voxel_centers) and current_y+1 < len(voxel_centers[0]) \
                and not checked_voxels[current_x+1][current_y+1][current_z] and \
                is_border_grain(voxel_centers, current_x+1, current_y+1, current_z, grain_id):
            queue.append((current_x+1, current_y+1, current_z))
            checked_voxels[current_x+1][current_y+1][current_z] = True
            
        if current_x+1 < len(voxel_centers) and current_y-1 > 0 \
                and not checked_voxels[current_x+1][current_y-1][current_z] and \
                is_border_grain(voxel_centers, current_x+1, current_y-1, current_z, grain_id):
            queue.append((current_x+1, current_y+1, current_z))
            checked_voxels[current_x+1][current_y+1][current_z] = True
            
        if current_x-1 > 0 and current_y+1 < len(voxel_centers[0]) \
                and not checked_voxels[current_x-1][current_y+1][current_z] and \
                is_border_grain(voxel_centers, current_x-1, current_y+1, current_z, grain_id):
            queue.append((current_x-1, current_y+1, current_z))
            checked_voxels[current_x-1][current_y+1][current_z] = True
            
        if current_x-1 > 0 and current_y-1 > 0 \
                and not checked_voxels[current_x-1][current_y-1][current_z] and \
                is_border_grain(voxel_centers, current_x-1, current_y-1, current_z, grain_id):
            queue.append((current_x-1, current_y-1, current_z))
            checked_voxels[current_x-1][current_y-1][current_z] = True
            
        # X and Z
        if current_x+1 < len(voxel_centers) and current_z+1 < len(voxel_centers[0][0]) \
                and not checked_voxels[current_x+1][current_y][current_z+1] and \
                is_border_grain(voxel_centers, current_x+1, current_y, current_z+1, grain_id):
            queue.append((current_x+1, current_y, current_z+1))
            checked_voxels[current_x+1][current_y][current_z+1] = True
            
        if current_x+1 < len(voxel_centers) and current_z-1 > 0 \
                and not checked_voxels[current_x+1][current_y][current_z-1] and \
                is_border_grain(voxel_centers, current_x+1, current_y, current_z-1, grain_id):
            queue.append((current_x+1, current_y, current_z-1))
            checked_voxels[current_x+1][current_y][current_z-1] = True
            
        if current_x-1 > 0 and current_z+1 < len(voxel_centers[0][0]) \
                and not checked_voxels[current_x-1][current_y][current_z+1] and \
                is_border_grain(voxel_centers, current_x-1, current_y, current_z+1, grain_id):
            queue.append((current_x-1, current_y, current_z+1))
            checked_voxels[current_x-1][current_y][current_z+1] = True
            
        if current_x-1 > 0 and current_z-1 > 0 \
                and not checked_voxels[current_x-1][current_y][current_z-1] and \
                is_border_grain(voxel_centers, current_x-1, current_y, current_z-1, grain_id):
            queue.append((current_x-1, current_y, current_z-1))
            checked_voxels[current_x-1][current_y][current_z-1] = True
            
        # Y and Z
        if current_y+1 < len(voxel_centers[0]) and current_z+1 < len(voxel_centers[0][0]) \
                and not checked_voxels[current_x][current_y+1][current_z+1] and \
                is_border_grain(voxel_centers, current_x, current_y+1, current_z+1, grain_id):
            queue.append((current_x, current_y+1, current_z+1))
            checked_voxels[current_x][current_y+1][current_z+1] = True
            
        if current_y+1 < len(voxel_centers[0]) and current_z-1 > 0 \
                and not checked_voxels[current_x][current_y+1][current_z-1] and \
                is_border_grain(voxel_centers, current_x, current_y+1, current_z-1, grain_id):
            queue.append((current_x, current_y+1, current_z-1))
            checked_voxels[current_x][current_y+1][current_z-1] = True
            
        if current_y-1 > 0 and current_z+1 < len(voxel_centers[0][0]) \
                and not checked_voxels[current_x][current_y-1][current_z+1] and \
                is_border_grain(voxel_centers, current_x, current_y-1, current_z+1, grain_id):
            queue.append((current_x, current_y-1, current_z+1))
            checked_voxels[current_x][current_y-1][current_z+1] = True
            
        if current_y-1 > 0 and current_z-1 > 0 \
                and not checked_voxels[current_x][current_y-1][current_z-1] and \
                is_border_grain(voxel_centers, current_x, current_y-1, current_z-1, grain_id):
            queue.append((current_x, current_y-1, current_z-1))
            checked_voxels[current_x][current_y-1][current_z-1] = True
            
        # X and Y and Z
        if current_x+1 < len(voxel_centers) and current_y+1 < len(voxel_centers[0]) and current_z+1 < len(voxel_centers[0][0]) \
                and not checked_voxels[current_x+1][current_y+1][current_z+1] and \
                is_border_grain(voxel_centers, current_x+1, current_y+1, current_z+1, grain_id):
            queue.append((current_x+1, current_y+1, current_z+1))
            checked_voxels[current_x+1][current_y+1][current_z+1] = True
            
        if current_x+1 < len(voxel_centers) and current_y+1 < len(voxel_centers[0]) and current_z-1 > 0 \
                and not checked_voxels[current_x+1][current_y+1][current_z-1] and \
                is_border_grain(voxel_centers, current_x+1, current_y+1, current_z-1, grain_id):
            queue.append((current_x+1, current_y+1, current_z-1))
            checked_voxels[current_x+1][current_y+1][current_z-1] = True
            
        if current_x+1 < len(voxel_centers) and current_y-1 > 0 and current_z+1 < len(voxel_centers[0][0]) \
                and not checked_voxels[current_x+1][current_y-1][current_z+1] and \
                is_border_grain(voxel_centers, current_x+1, current_y-1, current_z+1, grain_id):
            queue.append((current_x+1, current_y-1, current_z+1))
            checked_voxels[current_x+1][current_y-1][current_z+1] = True
            
        if current_x+1 < len(voxel_centers) and current_y-1 > 0 and current_z-1 > 0 \
                and not checked_voxels[current_x+1][current_y-1][current_z-1] and \
                is_border_grain(voxel_centers, current_x+1, current_y-1, current_z-1, grain_id):
            queue.append((current_x+1, current_y-1, current_z-1))
            checked_voxels[current_x+1][current_y-1][current_z-1] = True
            
        if current_x-1 > 0 and current_y+1 < len(voxel_centers[0]) and current_z+1 < len(voxel_centers[0][0]) \
                and not checked_voxels[current_x-1][current_y+1][current_z+1] and \
                is_border_grain(voxel_centers, current_x-1, current_y+1, current_z+1, grain_id):
            queue.append((current_x-1, current_y+1, current_z+1))
            checked_voxels[current_x-1][current_y+1][current_z+1] = True
            
        if current_x-1 > 0 and current_y+1 < len(voxel_centers[0]) and current_z-1 > 0 \
                and not checked_voxels[current_x-1][current_y+1][current_z-1] and \
                is_border_grain(voxel_centers, current_x-1, current_y+1, current_z-1, grain_id):
            queue.append((current_x-1, current_y+1, current_z-1))
            checked_voxels[current_x-1][current_y+1][current_z-1] = True
            
        if current_x-1 > 0 and current_y-1 > 0 and current_z+1 < len(voxel_centers[0][0]) \
                and not checked_voxels[current_x-1][current_y-1][current_z+1] and \
                is_border_grain(voxel_centers, current_x-1, current_y-1, current_z+1, grain_id):
            queue.append((current_x-1, current_y-1, current_z+1))
            checked_voxels[current_x-1][current_y-1][current_z+1] = True
            
        if current_x-1 > 0 and current_y-1 > 0 and current_z-1 > 0 \
                and not checked_voxels[current_x-1][current_y-1][current_z-1] and \
                is_border_grain(voxel_centers, current_x-1, current_y-1, current_z-1, grain_id):
            queue.append((current_x-1, current_y-1, current_z-1))
            checked_voxels[current_x-1][current_y-1][current_z-1] = True
    
    if min_distance_so_far == 999999999:
        raise RuntimeError('Grain boundary not found')
    return min_x, min_y, min_z, min_grain_id

In [43]:
def get_voxel_centers_index(x, y, z, voxel_size):
    return x // voxel_size, y // voxel_size, z // voxel_size

In [44]:
def get_xyz_from_voxel_centers_index(x, y, z, voxel_size):
    return x * voxel_size, y * voxel_size, z * voxel_size

In [45]:
def get_actual_grain_id(long_grain_id):
    return long_grain_id % 10000

In [17]:
# Save the centers of the voxels as a 3D array, with the values as the grain_ids
# Time: ~10 minutes
voxel_centers = get_voxel_centers_as_array(201, 376, 301, 2, '../data/voxel-centers.csv')
print('done')

Progress:  0.10000001758367279
Progress:  0.20000003516734557
Progress:  0.3000000087918364
Progress:  0.4000000263755092
Progress:  0.5
Progress:  0.6000000175836728
Progress:  0.7000000351673455
Progress:  0.8000000087918364
Progress:  0.9000000263755091
done


In [24]:
# Get the crack front points into a list
crack_front_points = get_crack_front_points('../data/crack_front_growth_rates_1500ppcf_transformed.csv')

In [119]:
# TODO: Verify this works as best I can with tests/vis

# Setup
voxel_size = 2
previously_looked_up = {}
crack_id_test = '1' # Use this to do each crack front iteratively

# Progress Printing
next_print = 3.15 / 100

with open('../data/nearest_grain_boundary.csv', 'a') as file:
    # Write the filer header the first time through
    if crack_id_test == '1':
        file.write('Crack ID, Theta, X, Y, Z, dadN3Dline, Nearest Grain ID, Grain Boundary X, Grain Boundary Y, Grain Boundary Z, Grain Boundary ID\n')
    for crack_front_point in crack_front_points:
#         print('natee0')
#     for crack_front_point in [('1',3.14,0,0,0,.333)]:
#     for crack_front_point in [('1',3.14,-0.93115, 332.85,306.54,.123)]:
        crack_id, theta, x, y, z, dadN3Dline = crack_front_point
        # If this is the crack front we want
        if crack_id_test == crack_id:
            x = float(x)
            y = float(y)
            z = float(z)

            # Get the nearest voxel_center for the x,y,z of the crack_front
            nearest_voxel_xyz = find_nearest_voxel_center(x, y, z, voxel_size)
            nearest_voxel_x, nearest_voxel_y, nearest_voxel_z = nearest_voxel_xyz
            print('natee1', x,y,z,nearest_voxel_xyz)
            if nearest_voxel_x < 0 or nearest_voxel_y < 0 or nearest_voxel_z < 0:
                continue
            
            # We need to keep track of what voxels we search
            checked_false = np.full(voxel_centers.shape, False, np.bool)

            # Get the grain_id of the voxel we're looking at
            voxel_centers_index_x, voxel_centers_index_y, voxel_centers_index_z = \
                get_voxel_centers_index(nearest_voxel_x, nearest_voxel_y, nearest_voxel_z, voxel_size)
            nearest_grain_id = get_actual_grain_id(voxel_centers[voxel_centers_index_x][voxel_centers_index_y][voxel_centers_index_z])

            # If we've already looked this voxel up
            if nearest_voxel_xyz in previously_looked_up:
                grain_boundary_index_x, grain_boundary_index_y, grain_boundary_index_z, grain_boundary_id = previously_looked_up[nearest_voxel_xyz]
            
            # Find the x,y,z,grain_id of the nearest grain boundary if we haven't looked it up already
            else:
                print('natee2')
                grain_boundary_index_x, grain_boundary_index_y, grain_boundary_index_z, grain_boundary_id = find_nearest_grain_boundary(voxel_centers,\
                  checked_false, voxel_centers_index_x, voxel_centers_index_y, voxel_centers_index_z, nearest_grain_id)
                print('natee3')
                previously_looked_up[nearest_voxel_xyz] = (grain_boundary_index_x, grain_boundary_index_y, grain_boundary_index_z, grain_boundary_id)

            # Go back to x,y,z from indices
            grain_boundary_x, grain_boundary_y, grain_boundary_z = get_xyz_from_voxel_centers_index(grain_boundary_index_x, \
                                                            grain_boundary_index_y, grain_boundary_index_z, voxel_size)

            temp = str(crack_id)+','+str(theta)+','+str(x)+','+str(y)+','+str(z)+','+str(dadN3Dline)+','+str(nearest_grain_id)+','+\
                str(grain_boundary_x)+','+str(grain_boundary_y)+','+str(grain_boundary_z)+','+str(grain_boundary_id)+'\n'
            file.write(temp)

            if float(theta) > next_print:
                print('Progress: ', float(theta) / 3.15)
                next_print += 3.15 / 100

print('done')

natee1 -0.93115 332.85 306.54 (0, 332, 306)
natee2
deque([(4, 166, 153), (0, 170, 153), (0, 164, 153), (0, 166, 165), (0, 166, 152)])
natee3
natee1 0.2469 330.57 306.95 (0, 330, 306)
natee2
deque([(3, 165, 153), (0, 170, 153), (0, 164, 153), (0, 165, 170), (0, 165, 152)])
natee3
Progress:  0.010638095238095238
natee1 0.32172 330.49 306.95 (0, 330, 306)
natee1 0.39684 330.4 306.99 (0, 330, 306)
natee1 0.47253 330.32 306.99 (0, 330, 306)
natee1 0.54891 330.24 306.99 (0, 330, 306)
natee1 0.624 330.16 307.04 (0, 330, 308)
natee2
deque([(7, 165, 154), (0, 171, 154), (0, 145, 154), (0, 165, 170), (0, 165, 152)])
natee3
natee1 0.70038 330.07 307.04 (0, 330, 308)
natee1 0.77712 329.99 307.04 (0, 330, 308)
natee1 0.85415 329.91 307.04 (0, 330, 308)
natee1 0.93155 329.83 307.04 (0, 330, 308)
natee1 1.0085 329.75 307.04 (2, 330, 308)
natee2
deque([(7, 165, 154), (1, 170, 154), (1, 145, 154), (1, 165, 201), (1, 165, 152)])
natee3
natee1 1.0868 329.67 307.15 (2, 330, 308)
natee1 1.1653 329.59 307.1

KeyboardInterrupt: 

2 0


In [ ]:
import numpy as np

def natee_search(gids_arr):
    gids = np.array(gids_arr)
    center_x = 0
    center_y = 0
    center_gid = gids[center_x][center_y]

    x_pos_feeler_x = center_x
    x_pos_feeler_y = center_y
    x_neg_feeler_x = center_x
    x_neg_feeler_y = center_y

    y_pos_feeler_x = center_x
    y_pos_feeler_y = center_y
    y_neg_feeler_x = center_x
    y_neg_feeler_y = center_y

    while x_pos_feeler_x+1 < len(gids) and gids[x_pos_feeler_x][x_pos_feeler_y] == center_gid:
        x_pos_feeler_x += 1

    while x_neg_feeler_x > 0 and gids[x_neg_feeler_x][x_neg_feeler_y] == center_gid:
        x_neg_feeler_x -= 1

    while y_pos_feeler_y+1 < len(gids[0]) and gids[y_pos_feeler_x][y_pos_feeler_y] == center_gid:
        y_pos_feeler_y += 1

    while y_neg_feeler_y > 0 and gids[y_neg_feeler_x][y_neg_feeler_y] == center_gid:
        y_neg_feeler_y -= 1

    def is_border_grain(center_gid, gids, x, y):
        if gids[x][y] == center_gid:
            return False
        if x+1 < len(gids) and gids[x+1][y] == center_gid:
            return True
        if x-1 >= 0 and gids[x-1][y] == center_gid:
            return True
        if y+1 < len(gids[0]) and gids[x][y+1] == center_gid:
            return True
        if y-1 >= 0 and gids[x][y-1] == center_gid:
            return True
        # Corners
        if x+1 < len(gids) and y+1 < len(gids[0]) and gids[x+1][y+1] == center_gid:
            return True
        if x-1 >= 0 and y+1 < len(gids[0]) and gids[x-1][y+1] == center_gid:
            return True
        if x+1 < len(gids) and y-1 < len(gids[0]) and gids[x+1][y-1] == center_gid:
            return True
        if x-1 < len(gids) and y-1 < len(gids[0]) and gids[x-1][y-1] == center_gid:
            return True
        return False

    from collections import deque

    checked = np.full(gids.shape, False, np.bool)
    queue = deque()

    used_x_pos = False
    used_x_neg = False
    used_y_pos = False
    used_y_neg = False

    if gids[x_pos_feeler_x][x_pos_feeler_y] != center_gid:
        queue.append((x_pos_feeler_x, x_pos_feeler_y))
        used_x_pos = True
    if gids[x_neg_feeler_x][x_neg_feeler_y] != center_gid:
        queue.append((x_neg_feeler_x, x_neg_feeler_y))
        used_x_neg = True
    if gids[y_pos_feeler_x][y_pos_feeler_y] != center_gid:
        queue.append((y_pos_feeler_x, y_pos_feeler_y))
        used_y_pos = True
    if gids[y_neg_feeler_x][y_neg_feeler_y] != center_gid:
        queue.append((y_neg_feeler_x, y_neg_feeler_y))
        used_y_neg = True

    min_distance_so_far = 99999999
    min_x = -999999999
    min_y = -999999999

    while len(queue) > 0:
        u = queue.popleft()
        current_x, current_y = u

        current_distance = ((current_y - center_y)**2 + (current_x - center_x)**2)**.5
        if current_distance < min_distance_so_far:
            min_distance_so_far = current_distance
            min_x = current_x
            min_y = current_y

        checked[current_x][current_y] = True

        if current_x+1 < len(gids) and not checked[current_x+1][current_y] and \
                        is_border_grain(center_gid, gids, current_x+1, current_y):
            queue.append((current_x+1, current_y))

        if current_x > 0 and not checked[current_x-1][current_y] and \
                        is_border_grain(center_gid, gids, current_x-1, current_y):
            queue.append((current_x-1, current_y))

        if current_y+1 < len(gids[0]) and not checked[current_x][current_y+1] and \
                        is_border_grain(center_gid, gids, current_x, current_y+1):
            queue.append((current_x, current_y+1))

        if current_y > 0 and not checked[current_x][current_y-1] and \
                        is_border_grain(center_gid, gids, current_x, current_y-1):
            queue.append((current_x, current_y-1))

    if min_distance_so_far == 99999999:
        raise RuntimeError('Did not find any grain boundaries...')
    print(min_x, min_y)

In [60]:
arr = \
[
    [0,0,0],
    [0,0,0],
    [1,0,1],
    [1,1,1]
]
pt = natee_search(arr)

1